# System of Equations

Let's motivate this section with  the __Lotka-Volterra equations__, also known as the predator-prey equations, are a pair of first-order nonlinear differential equations, frequently used to describe the dynamics of biological systems in which two species interact. The populations change through time according to the pair of equations:

$$
\dfrac{dx}{dt} = \alpha x - \beta xy \\
\dfrac{dy}{dt} = -\gamma y + \delta xy
$$

where
- $x$ is the number of prey;
- $y$ is the number of some predator;
- ${\tfrac {dy}{dt}}$ and ${\tfrac {dx}{dt}}$ represent the instantaneous growth rates of the two populations;
- $t$ represents time;
- $\alpha$, $\beta$, $\gamma$ and $\delta$ are positive real parameters describing the interaction of the two species.

## Physics-Informed Neural Networks

For years mathematicians and physicists are trying to model the world with differential equations. However, since the advent of techniques such as machine learning, neural networks and deep learning together with greater computing power, community has speculated that we could learn automatically (algorithms) anything with a enough amount of data. However, it seems this is not really true.

Philosophical Question: Could have a _machine_ discovered Newton's laws?

In 2019, Raissi, Perdikaris and Karniadakis introduced Physics-Informed Neural Networks (PINNs), neural networks that are trained to solve supervised learning tasks while respecting any given law of physics described by general nonlinear partial differential equations ([source](https://www.sciencedirect.com/science/article/pii/S0021999118307125)). PINNs are nowadays used to solve PDEs, fractional equations, and integral-differential equations.

PINNs approximate PDE solutions by training a neural network to minimize a loss function, including:

* Initial and boundary conditions along the space-time domain’s boundary
* PDE residual at selected points in the domain.

If you want to do a simplified analogy, initial and boundary conditions points will be an usual training dataset, but also it is necessary to embed physical laws (PDE) into the neural network.

PINNs can solve differential equations expressed, in the most general form, like:

$$
\begin{align*}
\mathcal{F}(u(z); \lambda) &= f(z) \quad z \text{ in } \Omega \\
\mathcal{B}(u(z)) &= g(z) \quad z \text{ in } \partial \Omega
\end{align*}
$$ 

defined on the domain $\Omega \subset \mathbb{R}^d$ with the boundary $\partial \Omega$. Where
- $z := (x_1, x_2, \ldots, t)^\top$ indicated the space-time coordinate vector,
- $u$ the unknown function,
- $\lambda$ the parameters related to the physics,
- $\mathcal{F}$ the non-linear differential operator,
- $f$ the function identifying the data of the problem,
- $\mathcal{B}$ the operator indicating arbitrary initial or boundary conditions, and
- $g$ the boundary function.

In the PINN methodology, $u(z)$ is computationally predicted by a NN, parametrized by a set of parameters $\theta$, giving rise to an approximation
$$
\hat{u}_\theta(z) \approx u(z)
$$

The optimization problem we want to deal with it is

$$
\min_\theta \; \omega_\mathcal{F} \mathcal{L}_\mathcal{F}(\theta) + \omega_\mathcal{B} \mathcal{L}_\mathcal{B}(\theta) + \omega_{\text{data}} \mathcal{L}_{\text{data}}(\theta)
$$

this is three weighted loss functions, each one depending on
- $\mathcal{L}_\mathcal{F}$, differential equation, 
- $\mathcal{L}_\mathcal{B}$, boundary conditions, and
- $\mathcal{L}_{\text{data}}$, (eventually) some known data.

![PINNs](https://media.springernature.com/full/springer-static/image/art%3A10.1007%2Fs10915-022-01939-z/MediaObjects/10915_2022_1939_Fig2_HTML.png?as=webp)

[Source](https://link.springer.com/article/10.1007/s10915-022-01939-z)

## PINNs solving ODEs

For our Lotka-Volterra system the solution $u$ will be a vector such that 
$$
u(t) = (x(t), y(t))^\top
$$
and there are not boundary conditions, only initial conditions.

We want to train a network that looks like this

![lotka_volterra_pinn](../images/lotka_volterra_pinn.png)

## Import packages

The most important package is `deepxde`, which allows us to implement Physic-Informed Neural Networs approaches with a few lines of code. We will need `numpy` for array operations, `matplotlib` for visualizations and `scipy` for getting the real solution (which is not always possible, if not we would only use this approach).

In [ ]:
# !pip install deepxde  # Run this line if you are in Google Colab

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import deepxde as dde

from scipy.integrate import solve_ivp
from deepxde.backend import tf

## ODE Residuals

Since we are trying to embed the physics onto the neural networks we need to define 

In [ ]:
alpha = 2 / 3
beta = 4 / 3
gamma = 1
delta = 1

In [ ]:
def ode(t, Y):
    x = Y[:, 0:1]
    y = Y[:, 1:2]

    dx_dt = dde.grad.jacobian(Y, t, i=0)
    dy_dt = dde.grad.jacobian(Y, t, i=1)
    
    return [
        dx_dt - alpha * x + beta * x * y,
        dy_dt + gamma * y  - delta * x * y
    ]

Here `t` is the indepent variable and `Y` is an array with two columns (since our system considers two equations). To define the first derivative is as easy as using `dde.grad.jacobian`, just be sure about the component `i`, in this case we decided `i=0` corresponds to the variable $x(t)$ and `i=1` to $y(t)$.

## Initial conditions

It is important to decide a time invertal where we will working on. As an example let's consider between $t=0$ and $t=1$.

In [ ]:
t_initial = 0
t_final = 10

And now we need to declare this element for our neural network, if not, the algorithm wouldn't know where to make the estimations.

In [ ]:
geom = dde.geometry.TimeDomain(t_initial, t_final)

Then we have to create a function for defining boundaries, since our geometry it is only on time we will use the default one, don't worry about it.

In [ ]:
def boundary(_, on_initial):
    return on_initial

Let's choose initial conditions for $x$ and $y$

In [ ]:
x0 = 1.2
y0 = 0.8

And then we have to tell to our algorithm these are the initial conditions for the learning process

In [ ]:
ic_x = dde.icbc.IC(geom, lambda x: x0, boundary, component=0)
ic_y = dde.icbc.IC(geom, lambda x: y0, boundary, component=1)

## Data object

Everything related to the differential equations and initial conditions has to be inside a new object `dde.data.PDE` (do not worry, it also consider systems of ordinary differential equations).

In [ ]:
data = dde.data.PDE(
    geom,
    ode,
    [ic_x, ic_y],
    num_domain=512,
    num_boundary=2
)

In order to test our model we need more points, we considered 512 points inside our domain with `num_domain=512`. Finally, since we are working on a time domain there are only two points in its boundary (`num_boundary=2`).

## Neural Network

It is time for choosing a neural network architecture. For simplicity, we will use a Fully-connected neural network (`dde.nn.FNN'). The most important things are:

- Input layer (the first one) needs only one node/neuron since our indepent variable is only time $t$.
- The output layer (the last one) nneds two nodes/neurons since we are working on a system of two equations.

Do not worry so much about the amount of layers or neurons in each hidden layer, as a rule of thumb error should decrease while you add more layers and neurons, but it will take more computational time. Activation functions and the initializer are more parameters the user must choose, usually `Glorot normal` works well as initializer. However, we would recommend you to try different activation functions, for example, `relu`, `sigmoid` or `swish`.

In [ ]:
neurons = 64
layers = 6
activation = "tanh"
initializer = "Glorot normal"
net = dde.nn.FNN([1] + [neurons] * layers + [2], activation, initializer)

## Model

The library we are working with needs put everything together in a new object, but it is just one line of code.

In [ ]:
model = dde.Model(data, net)

## Training

For training we will go with an Adam optimizer (a very popular one nowadays) and a learning rate of 0.001 (smaller learning rates may give you better results but it will take many more iterations).

Just for simplicity we will take 50000 iterations, but another rule of thumb it is that as you increase the number of iterations the loss value should decrease as well.

In [ ]:
model.compile("adam", lr=0.001)
losshistory, train_state = model.train(iterations=50000)

In [ ]:
# model.compile("L-BFGS")
# losshistory, train_state = model.train()

We can plot the loss history with a simple command.

In [ ]:
dde.utils.external.plot_loss_history(losshistory)

## Compare

Now it is time to compare our model. We chose the Lotka-Volterra because we can obtain the solution with a Runge-Kuta algorithm, for instance, we used `solve_ivp` from `scipy`.

In [ ]:
def generate_data(
    t,
    x0,
    y0,
    alpha,
    beta,
    gamma,
    delta
):

    def func(t, Y):
        x, y = Y
        dx_dt = alpha * x - beta * x * y
        dy_dt = - gamma * y  + delta * x * y
        return dx_dt, dy_dt

    Y0 = [x0, y0]
    t_span = (t[0], t[-1])
    sol = solve_ivp(func, t_span, Y0, t_eval=t)
    x_true, y_true = sol.y
    return x_true, y_true

Let's generate a fine grid of time, generate data with Runge-Kuta and then compare both approaches in the same plot.

In [ ]:
t = np.linspace(t_initial, t_final, 100)
x_true, y_true = generate_data(t, x0, y0, alpha, beta, gamma, delta)
plt.plot(t, x_true, color="green", label="x_true")
plt.plot(t, y_true, color="blue", label="y_true")

sol_pred = model.predict(t.reshape(-1, 1))
x_pred = sol_pred[:, 0:1]
y_pred = sol_pred[:, 1:2]

plt.plot(t, x_pred, color="red", linestyle="dashed", label="x_pred")
plt.plot(t, y_pred, color="orange", linestyle="dashed", label="y_pred")
plt.legend()
plt.show()

As you can see both algorithms gave us almost identical results. One of the pros that we would like to point out about Physics-Informed neural networks is that for more complex systems you only need to change a very few things more, specifically residuals. Most of the numerical work is done automatically by machine learning libraries as `TensorFlow`, `Torch`, `JAX`, etc. so it is easy to scale it up, even better when we can take advantage of GPUs. For you, as an user, your challenge will be in pick suitable hyper-parameters (number of layers, number of neurons, activation function, number of iterations, etc.) but this also could be done by other algorithms, however these are out of the scope of this lesson.